In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings
import json
import pickle
import re
import os
import tarfile
import shutil

In [2]:
# First we need to exctract the chroma db (which is too large for Github. You should update the chroma tarfile and supress the sqlite file if you wan't to update it)
#      open a shell :
#           tar - xf  path/to/file/to/tar path/to/file/export
with tarfile.open("./chroma/chroma_sql.tar", "r", encoding = "utf-8") as tar:
    tar.extract("./chroma.sqlite3")
shutil.move(src="chroma.sqlite3", dst="./chroma/chroma.sqlite3")

'./chroma/chroma.sqlite3'

In [3]:
with open("api_key", "r", encoding="utf-8") as key:
    api_key = key.read()

with open("HF_api_key", "r", encoding="utf-8") as key:
    HF_api_key = key.read()

orga_key = "org-mqd2akvmI5TNokavyM3mW"

embeddings_model_name="all-MiniLM-L6-v2"

loaders = []
docs = []

path = "../data/Texte/"
listdir = os.listdir(path)
listdir.sort(key=lambda x:int(x))

for i, folder in enumerate(listdir):
    print(f"\r{i} / {len(listdir)}", end = "")
    f_path = os.path.join(path, folder)
    for file in os.listdir(f_path):
        if file == "full.txt":
            file_path = os.path.join(f_path, file)
            loaders.append(TextLoader(file_path, encoding="utf-8"))
            
for i, loader in enumerate(loaders):
    print(f"\r{i} / {len(loaders)}", end = "")
    docs.extend(loader.load())

def out_prompt(out):
    i = 0
    j = 0
    while j < len(out):
        charac = out[j]
        if charac == ".":
            if re.search("etc", out[j-5:j]):
                print(charac, end = "")
                pass
            else:
                i = 0
                print(charac, "\n\n", end = "")
                j += 1
        elif i >= 100:
            if charac != " ":
                print(charac, end = "")
            else:
                print("\n", end = "")
                i = 0
        else:
            print(charac, end = "")
        i += 1
        j += 1

            
# Sentence_transformers
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name, model_kwargs={'device' : 'cpu'})


# Llama2 (Couldn't create embeddings for now (more than a day of computing))
n_gpu_layers = 35  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
#llama = LlamaCppEmbeddings(model_path="./llama-2-7b-chat.ggmlv3.q4_0.bin", n_ctx = 4096, f16_kv=True, n_gpu_layers=n_gpu_layers, n_batch=n_batch)

#OpenAI ?

2773 / 2774

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=300, length_function = len, add_start_index = True)
documents = text_splitter.split_documents(docs)

print(f" Nombre de documents : {len(documents)}")

vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory="./chroma")
#save_embeddings(vectorstore, "embeddings.pkl")

 Nombre de documents : 8838


KeyboardInterrupt: 

In [4]:
vectorstore2 = Chroma(persist_directory = "./chroma", embedding_function=embeddings)
n_gpu_layers = 20
n_batch = 512
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
# llm = LlamaCpp(
#     model_path="./llama-2-7b-chat.ggmlv3.q4_0.bin",
#     n_gpu_layers=n_gpu_layers,
#     n_batch=n_batch,
#     n_ctx=2048, 
#     f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
#     callback_manager=callback_manager,
#     verbose=True,
# )

In [5]:
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate


def chat(llm_info = None, kMos = 100):
    
    user_message = input('Quelle questinon souhaitez vous poser ? :\n')
    vectorstore2 = Chroma(persist_directory = "./chroma", embedding_function=embeddings)

    llmOAI = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo-16k")

    prompt_template = """
    Utilise uniquement les informations données en contexte pour répondre de manières longues et détaillées aux questions posées.
    Cite des passages des informations en contexte pour appuyer tes réponses. N'invente pas de citation si tu ne cite pas précisément les informations en contexte
    N'essaye pas d'inventer de réponses si tu ne trouve pas la réponse.
    Propose des réponses points par points.
    {context}
    Question: {question}"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context","question"]
    )

    chain_type_kwargs = {"prompt": PROMPT}

    match llm_info:
        case "MOSAIC":
            
            repo_id = "mosaicml/mpt-30b-chat"

            llmMos = HuggingFaceHub(
                huggingfacehub_api_token = HF_api_key,
                repo_id = repo_id
            )

            user_message = "qu'est-ce que l'Institut des Sciences de l'Homme (ISH) ?"
            # "Quels sont les principaux choix stratégiques apportés par Millénaire 3 ?"

            score_threshold = 0.2
                
            qa = RetrievalQA.from_chain_type(llm=llmMos, 
                                        chain_type="stuff", 
                                        retriever=vectorstore2.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": kMos, "score_threshold" : score_threshold}), 
                                        return_source_documents=True, 
                                        chain_type_kwargs=chain_type_kwargs
                                        )
            out = qa({"query" : user_message})

        case "OAI":
            
                score_threshold = 0.2

                while True:
                        
                    qa = RetrievalQA.from_chain_type(llm=llmOAI, 
                                                chain_type="stuff", 
                                                retriever=vectorstore2.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 30, "score_threshold" : score_threshold}), 
                                                return_source_documents=True, 
                                                chain_type_kwargs=chain_type_kwargs
                                                )
                    try:
                        out = qa({"query" : user_message})
                        print(f"---------- COS : {round(score_threshold, 2)} --------------")
                        break
                    except:
                        print("UPDATE THRESHOLD")
                        score_threshold += 0.05
    return(out)

In [7]:
out = chat("OAI", kMos = 150)
out_prompt(out["result"])
print("\n ----------------- DOCS ------------------\n")
for doc in out["source_documents"]:
    print(doc)

---------- COS : 0.2 --------------
La santé à Lyon est un domaine important qui attire de nombreux professionnels de la santé et chercheurs
dans la région. 

Selon la synthèse du groupe santé de Lyon 2020, il est mentionné que "Lyon est une ville qui se distingue
par son dynamisme dans le secteur de la santé". 

Cela indique que la ville de Lyon est reconnue pour son engagement et son investissement dans le domaine
de la santé. 


De plus, la présence de CPE-Lyon dans l'enseignement supérieur lyonnais renforce également l'importance
de la santé à Lyon. 

CPE-Lyon est une école d'ingénieurs spécialisée dans les domaines de la chimie, de la physique et de
l'énergie. 

Cette école contribue à la recherche et à l'innovation dans le domaine de la santé, en formant des étudiants
qui deviendront les futurs professionnels de la santé. 


En ce qui concerne la recherche, Lyon est également reconnue pour ses atouts dans le domaine de la
santé. 

La région lyonnaise compte de nombreux instituts 

In [155]:
from langchain.document_transformers import (
    LongContextReorder,
)
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate

query = "l'emploi à Lyon comparé à d'autres métropole européenne"

retriever = vectorstore2.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 50, "score_threshold" : 0.3})
docs = retriever.get_relevant_documents(query)
# Reorder the documents:
# Less relevant document will be at the middle of the list and more
# relevant elements at begining / end.
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)


document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
### llm = OpenAI()
stuff_prompt_override = """Given this text extracts:
-----
{context}
-----
Please answer the following question:
{query}"""
prompt = PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
)

# Instantiate the chain
llm_chain = LLMChain(llm=llmOAI, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)
chain.run(input_documents=reordered_docs, query=query)


'There is no information provided in the given text extracts about employment in Lyon compared to other European cities.'

In [5]:
vectorstore2.get()["ids"]

['4551d6a2-3200-11ee-b126-9234d372d0bb',
 '4551d8d2-3200-11ee-b126-9234d372d0bb',
 '4551d936-3200-11ee-b126-9234d372d0bb',
 '4551d986-3200-11ee-b126-9234d372d0bb',
 '4551d9d6-3200-11ee-b126-9234d372d0bb',
 '4551da26-3200-11ee-b126-9234d372d0bb',
 '4551da6c-3200-11ee-b126-9234d372d0bb',
 '4551dab2-3200-11ee-b126-9234d372d0bb',
 '4551db16-3200-11ee-b126-9234d372d0bb',
 '4551db70-3200-11ee-b126-9234d372d0bb',
 '4551dbca-3200-11ee-b126-9234d372d0bb',
 '4551dc1a-3200-11ee-b126-9234d372d0bb',
 '4551dc6a-3200-11ee-b126-9234d372d0bb',
 '4551dcba-3200-11ee-b126-9234d372d0bb',
 '4551dd14-3200-11ee-b126-9234d372d0bb',
 '4551dd96-3200-11ee-b126-9234d372d0bb',
 '4551dde6-3200-11ee-b126-9234d372d0bb',
 '4551de22-3200-11ee-b126-9234d372d0bb',
 '4551de7c-3200-11ee-b126-9234d372d0bb',
 '4551deb8-3200-11ee-b126-9234d372d0bb',
 '4551df12-3200-11ee-b126-9234d372d0bb',
 '4551df4e-3200-11ee-b126-9234d372d0bb',
 '4551df94-3200-11ee-b126-9234d372d0bb',
 '4551dfe4-3200-11ee-b126-9234d372d0bb',
 '4551e020-3200-

# nettoyage corpus

In [34]:
tmp = []
for i, folder in enumerate(listdir):
    #print(f"\r{i} / {len(listdir)}", end = "")
    f_path = os.path.join(path, folder)
    if i != int(folder):
        print(i, folder)
    for file in os.listdir(f_path):
        if file == "full.txt":
            file_path = os.path.join(f_path, file)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
                tmp.append(text)
            # with open(file_path, "w", encoding="utf-8") as f:
            #     f.write(text)

795 796
796 797
797 798
798 799
799 800
800 801
801 802
802 803
803 804
804 805
805 806
806 807
807 808
808 809
809 810
810 811
811 812
812 813
813 814
814 815
815 816
816 817
817 818
818 819
819 820
820 821
821 822
822 823
823 824
824 825
825 826
826 827
827 828
828 829
829 830
830 831
831 832
832 833
833 834
834 835
835 836
836 837
837 838
838 839
839 840
840 841
841 842
842 843
843 844
844 845
845 846
846 847
847 849
848 850
849 851
850 852
851 853
852 854
853 855
854 856
855 857
856 858
857 859
858 860
859 861
860 862
861 863
862 864
863 865
864 866
865 867
866 868
867 869
868 870
869 871
870 872
871 873
872 874
873 875
874 876
875 877
876 878
877 879
878 880
879 881
880 882
881 883
882 884
883 885
884 886
885 887
886 888
887 889
888 890
889 891
890 892
891 893
892 894
893 895
894 896
895 897
896 898
897 899
898 900
899 901
900 902
901 903
902 904
903 905
904 906
905 907
906 908
907 909
908 910
909 911
910 912
911 913
912 914
913 915
914 916
915 917
916 918
917 919
918 920
919 921


In [35]:
for i, text in enumerate(tmp):
    print(f"------------- {i} -------------")
    print(text[:200])
    

------------- 0 -------------
Action publique : comment penser la solidarité depuis un territoire ?
Qu’ont en commun le dérèglement climatique, l’inflation, le vieillissement de la population, les failles du système de santé, le h
------------- 1 -------------
Vers une reconquête de la qualité de l’air : pourquoi, comment ?
Changer nos modes de déplacement et retrouver un air de qualité. Telles sont les ambitions de la mise en place des « Zones à Faibles Ém
------------- 2 -------------
Résilience : quelles articulations entre acteurs publics et initiatives du territoire ?
Ces dernières années, la question de la résilience des territoires s’est installée dans l’actualité face à la mu
------------- 3 -------------
L’inclusion, un nouveau modèle de société ?
La notion d’inclusion a renouvelé le modèle dit d’intégration des années 1980. Jean-Marc Berthet explique qu’avec elle « ce n’est plus aux individus à déplo
------------- 4 -------------
Économie circulaire : au-delà du recyclage, co